# COMP 3610 – A3

- Zidane Timothy, Maia Neptune, Christophe Gittens

In [6]:
from pathlib import Path
import os
import tarfile
import pandas as pd
import shutil
import glob

import time, matplotlib.pyplot as plt, seaborn as sns, matplotlib.ticker as ticker
import numpy as np


from datasets import load_dataset, load_from_disk
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import json

Load clean and merge respective partioned categories

In [ ]:
# Load all cleaned pkl files
def load_all_data(pkl_directory):
    files = glob.glob(f"{pkl_directory}/*.bz2")
    for files in pkl_directory:
        mk_pkl = load_from_disk(pkl_directory)
        pd.concat(mk_pkl, ignore_index=True)

    # files = glob.glob(f"{pkl_directory}/*.pkl.bz2")
    # dfs = [pd.read_pickle(f) for f in files]
    # return pd.concat(dfs, ignore_index=True)


    

In [10]:
start_path = r"D:\UWI\Year 3\Sem 2\COMP3610-Big-Data\Assignments\Assignment#3\A3\datasets\output_folder\cleaned"

all_pkls = load_all_data(start_path)